In [ ]:
import shutil
import random
import glob
import os
names = ["A","B","C"]
os.makedirs("./test", exist_ok=True)

for name in names:
    in_dir = "./face/"+name+"/*"
    in_jpg=glob.glob(in_dir)
    img_file_name_list=os.listdir("./face/"+name+"/")
    random.shuffle(in_jpg)
    os.makedirs('./test/' + name, exist_ok=True)
    for t in range(len(in_jpg)//5):
        shutil.move(str(in_jpg[t]), "./test/"+name)

In [ ]:
import os
import cv2
import glob
from scipy import ndimage
"""
faceディレクトリから画像を読み込んで回転、ぼかし、閾値処理をしてtrainディレクトリに保存する.
"""
names = ["A","B","C"]
os.makedirs("./train", exist_ok=True)
for name in names:
    in_dir = "./face/"+name+"/*"
    out_dir = "./train/"+name
    os.makedirs(out_dir, exist_ok=True)
    in_jpg=glob.glob(in_dir)
    img_file_name_list=os.listdir("./face/"+name+"/")
    for i in range(len(in_jpg)):
        img = cv2.imread(str(in_jpg[i]))
        for ang in [-10,0,10]:
            img_rot = ndimage.rotate(img,ang)
            img_rot = cv2.resize(img_rot,(64,64))
            fileName=os.path.join(out_dir,str(i)+"_"+str(ang)+".jpg")
            cv2.imwrite(str(fileName),img_rot)
            # 閾値
            img_thr = cv2.threshold(img_rot, 100, 255, cv2.THRESH_TOZERO)[1]
            fileName=os.path.join(out_dir,str(i)+"_"+str(ang)+"thr.jpg")
            cv2.imwrite(str(fileName),img_thr)
            # ぼかし
            img_filter = cv2.GaussianBlur(img_rot, (5, 5), 0)
            fileName=os.path.join(out_dir,str(i)+"_"+str(ang)+"filter.jpg")
            cv2.imwrite(str(fileName),img_filter)

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import cv2, time
from datetime import datetime
import slackweb
import os

#proxy認証
#os.environ["HTTP_PROXY"] = "proxy_adress"
#os.environ["HTTPS_PROXY"] = "proxy_adress"

# 正面の顔検出用
cascPath = '-----Your_Path-----/lbpcascades/lbpcascade_frontalface.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

#Slackに通知を飛ばす
#slack = slackweb.Slack(url="<https://hooks.slack.com/services/-----Your_adress-----")

#MJPEGStream
URL = "-----raspberry_address------/?action=stream"
s_video = cv2.VideoCapture(URL)

while True:
  ret, img = s_video.read()

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.equalizeHist( gray )
  faces = faceCascade.detectMultiScale(gray, 1.1, 3, 0, (10, 10))

  for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
      time = datetime.now().strftime('%Y%m%d%H%M%S')
      # imgフォルダに格納
      path = '.\\\\img\\\\' + time + '.jpg'
      cut_face = img[y:y+h,x:x+w,:]
      cut_face_resize = cv2.resize(cut_face,(64,64))
      cv2.imwrite(path, cut_face_resize)
      # slackに通知
      # slack.notify(text="メッセージ")
    
  cv2.imshow("Face-Detect-Video",img)
  key = cv2.waitKey(1) & 0xff
  if key == ord('q'): break